In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import statsmodels
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import os
%matplotlib inline

In [2]:
# os.chdir('/home/mike/PycharmProjects/WiFinder/Data/final_csvs')

In [3]:
df = pd.read_csv('merged_table.csv')

In [4]:
df.head()

,Associated_count,Time,ID,Room_Capacity,Module,Registered_Students,Hour,Date,Room,Occupancy
0,2,09:02:20,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0
1,1,09:07:24,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0
2,0,09:12:01,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0
3,0,09:16:53,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0
4,1,09:21:55,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0


In [5]:
# replace NAs in registered students

df['Registered_Students'] = df['Registered_Students'].fillna(0)

In [6]:
# create new float column to house adjusted count

df['Adjusted_Count'] = 0.0

In [7]:
# create new feature, proportion of associated count connections to room capacity

for row in range(df.shape[0]):
    assoc_count = df.columns.get_loc('Associated_count')
    adj_count = df.columns.get_loc('Adjusted_Count')
    room_cap = df.columns.get_loc('Room_Capacity')

    count_ratio = (df.iat[row, assoc_count] / df.iat[row, room_cap])
    df.iat[row, adj_count] = count_ratio

In [8]:
# create new columns to map dictionaries onto

df['Avg_Count_60min'] = df['ID']
df['Avg_Count_30min'] = df['ID']
df['Avg_Adj_Count_60min'] = df['ID']
df['Avg_Adj_Count_30min'] = df['ID']

In [9]:
# groupby to pull average features of each class

hours = df.groupby('ID')

In [10]:
# compute average associated count over 60 minutes

average_assoc_count = hours['Associated_count'].mean()

average_assoc_count = average_assoc_count.to_dict()

In [11]:
# compute average adjusted count over 60 minutes

average_adjusted_count = hours['Adjusted_Count'].mean()

average_adjusted_count = average_adjusted_count.to_dict()

In [12]:
# select only times within h:15 and h:45 to cut out spikes before and after classes

df['minute'] = df['Time']
df['minute'] = pd.to_datetime(df['minute'])
#just keep the minute of time
df['minute'] = df['minute'].apply(lambda x: x.strftime('%M'))
df_30mins = df.loc[(df['minute'] > '15') & (df['minute'] < '45')]
#made new data frame called df_1

BREAK LADS

In [13]:
half_hours = df_30mins.groupby('ID')

In [14]:
# (half_hours['Adjusted_Count'].mean() - half_hours['Adjusted_Count'].mean()).median()

In [15]:
# compute average associated count over 30 minutes

average_assoc_count_30mins = half_hours['Associated_count'].mean()

average_assoc_count_30mins = average_assoc_count_30mins.to_dict()

In [16]:
# compute average adjusted count over 30 minutes

average_adjusted_count_30mins = half_hours['Adjusted_Count'].mean()

average_adjusted_count_30mins = average_adjusted_count_30mins.to_dict()

In [17]:
df = df.replace({'Avg_Count_60min': average_assoc_count})

In [18]:
df = df.replace({'Avg_Count_30min': average_assoc_count_30mins})

In [19]:
df = df.replace({'Avg_Adj_Count_60min': average_adjusted_count})

In [20]:
df = df.replace({'Avg_Adj_Count_30min': average_adjusted_count_30mins})

In [21]:
df.head(1)

,Associated_count,Time,ID,Room_Capacity,Module,Registered_Students,Hour,Date,Room,Occupancy,Adjusted_Count,Avg_Count_60min,Avg_Count_30min,Avg_Adj_Count_60min,Avg_Adj_Count_30min,minute
0,2,09:02:20,92015-11-03B002,90,Vacant,-1,9,2015-11-03,B002,0,0.022222,4.75,2,0.052778,0.022222,02


In [22]:
columns_to_drop = ['Time', 'Associated_count', 'ID', 'minute', 'Adjusted_Count']

# # df = df.drop('Time', axis=1)
# # df = df.drop('Associated_count', axis=1)
# df = df.drop('ID', axis=1)
# df = df.drop('', axis=1)
# df = df.drop('ID', axis=1)

for column in columns_to_drop:
    if column in df.columns:
        df = df.drop('{}'.format(column), axis=1)
    else:
        print(column, ' is not present in DataFrame')

In [23]:
df.head(10)

,Room_Capacity,Module,Registered_Students,Hour,Date,Room,Occupancy,Avg_Count_60min,Avg_Count_30min,Avg_Adj_Count_60min,Avg_Adj_Count_30min
0,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
1,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
2,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
3,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
4,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
5,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
6,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
7,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
8,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222
9,90,Vacant,-1,9,2015-11-03,B002,0,4.75,2,0.052778,0.022222


In [24]:
df = df.drop_duplicates()

In [25]:
df = df.reset_index(drop=True)

In [27]:
df.to_csv('ABT.csv', index=False)